In [0]:
from pyspark.sql.functions import *

amazon_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv",header = True, inferSchema = True)
amazon_df = amazon_df.withColumn("Date", to_date(col("InvoiceDate")))
amazon_df = amazon_df.withColumn('Year',year(col("Date")))
amazon_df = amazon_df.withColumn('Month',month(col("Date")))
amazon_df = amazon_df.withColumn('Day',date_format(col("Date"), "d"))
amazon_df = amazon_df.withColumn('quarter',quarter(amazon_df.InvoiceDate))
amazon_df.dropna()
amazon_df.createOrReplaceTempView("amazontable")
amazon_df.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+--------------------+--------------+-------------+------------+---------+----+----+----+----+----------+----+-----+---+-------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|         InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|_c15|_c16|_c17|_c18|      Date|Year|Month|Day|quarter|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+--------------------+--------------+-------------+------------+---------+----+----+----+----+----------+----+-----+---+-------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|       1470|              20|                30|   

#1.The particular inventory you need to store at a specific location

In [0]:
#a. Find the top five performing mobile brands for the months of November, December and January r each year.
spark.sql("select Brand, Year, Month, sum(Quantity) as ProductSold from amazontable where SubCategory='mobile' and Month in (11,12,1) group by Year, Month, Brand order by ProductSold DESC").show()
#Samsung, Blu, Apple, Nokia, LG

+-------+----+-----+-----------+
|  Brand|Year|Month|ProductSold|
+-------+----+-----+-----------+
|samsung|2015|   12|      490.0|
|samsung|2015|   11|      434.0|
|samsung|2017|    1|      416.0|
|samsung|2016|    1|      402.0|
|samsung|2016|   12|      383.0|
|samsung|2017|   12|      381.0|
|samsung|2015|    1|      377.0|
|samsung|2018|    1|      377.0|
|samsung|2016|   11|      376.0|
|samsung|2017|   11|      306.0|
|    blu|2015|   11|      234.0|
|    blu|2015|   12|      225.0|
|  apple|2016|    1|      225.0|
|  apple|2015|    1|      223.0|
|    blu|2016|   12|      207.0|
|  nokia|2016|    1|      198.0|
|    blu|2016|    1|      195.0|
|  apple|2015|   11|      193.0|
|    blu|2016|   11|      187.0|
|    blu|2017|   12|      183.0|
+-------+----+-----+-----------+
only showing top 20 rows



In [0]:
#b. Find the top five selling products for the months of November, December and January for each year.
spark.sql("select Category, Year, Month, sum(Quantity) as ProductSold from amazontable where Month in (11,12,1) group by Category,Year, Month order by ProductSold DESC").show()

+-----------+----+-----+-----------+
|   Category|Year|Month|ProductSold|
+-----------+----+-----+-----------+
|   clothing|2017|   11|    13450.0|
|   clothing|2017|   12|    12286.0|
|   clothing|2015|   12|    11865.0|
|   clothing|2016|    1|    11859.0|
|   clothing|2016|   12|    11510.0|
|   clothing|2015|    1|    11243.0|
|   clothing|2015|   11|    10464.0|
|   clothing|2016|   11|    10142.0|
|   clothing|2017|    1|    10060.0|
|   clothing|2018|    1|     9865.0|
|electronics|2015|   12|     3240.0|
|electronics|2015|   11|     3086.0|
|electronics|2016|    1|     2948.0|
|electronics|2016|   11|     2902.0|
|electronics|2016|   12|     2809.0|
|electronics|2017|    1|     2703.0|
|electronics|2015|    1|     2541.0|
|electronics|2017|   11|     2491.0|
|electronics|2017|   12|     2468.0|
|electronics|2018|    1|     2467.0|
+-----------+----+-----+-----------+
only showing top 20 rows



#2. Give valuable insights to the investors

In [0]:
#a. Number of products sold along with revenue aggregated quarterly
spark.sql("select Year, quarter,sum(Quantity) as ProductSold,sum(TransactionAmount) as Total_Revenue from amazontable where Year < 2022 and Year > 2000 group by Year, quarter order by Year,Total_Revenue").show()



+----+-------+-----------+---------------+
|Year|quarter|ProductSold|  Total_Revenue|
+----+-------+-----------+---------------+
|2015|      1|    42496.0|      307081579|
|2015|      3|    41699.0|      310227925|
|2015|      4|    46055.0|      323594733|
|2015|      2|    41987.0|223020301919487|
|2016|      1|    48914.0|223020347532586|
|2016|      4|    42262.0|446040317719413|
|2016|      2|    41919.0|446040328695161|
|2016|      3|    45229.0|669060342716527|
|2017|      3|    40215.0|      291984175|
|2017|      1|    44753.0|      323024597|
|2017|      2|    44509.0|      345596349|
|2017|      4|    45181.0|446040345736675|
|2018|      2|     4143.0|       32793936|
|2018|      1|    42288.0|669060299009857|
+----+-------+-----------+---------------+



In [0]:
#b. Number of products sold in each category aggregated quarterly
spark.sql("select Category, Year, quarter,sum(Quantity) as ProductSold from amazontable where Year < 2022 and Year > 2000 group by Category, Year, quarter order by Category, Year").show()

+-------------+----+-------+-----------+
|     Category|Year|quarter|ProductSold|
+-------------+----+-------+-----------+
|apple carplay|2015|      1|        4.0|
|apple carplay|2016|      4|        4.0|
|apple carplay|2016|      1|        4.0|
|apple carplay|2017|      1|        4.0|
|apple carplay|2017|      4|        8.0|
|apple carplay|2017|      2|        4.0|
|apple carplay|2018|      1|        8.0|
|     clothing|2015|      1|    32631.0|
|     clothing|2015|      2|    32437.0|
|     clothing|2015|      4|    34226.0|
|     clothing|2015|      3|    31967.0|
|     clothing|2016|      2|    31538.0|
|     clothing|2016|      1|    38581.0|
|     clothing|2016|      3|    34235.0|
|     clothing|2016|      4|    31830.0|
|     clothing|2017|      3|    30400.0|
|     clothing|2017|      1|    34113.0|
|     clothing|2017|      4|    35182.0|
|     clothing|2017|      2|    33725.0|
|     clothing|2018|      1|    32801.0|
+-------------+----+-------+-----------+
only showing top

In [0]:
#c. A number of products sold for each brand aggregated quarterly.
spark.sql("select Brand, Year, quarter,sum(Quantity) as ProductSold from amazontable where Year < 2022 and Year > 2000 group by Brand, Year, quarter order by Brand, Year").show()

+--------------------+----+-------+-----------+
|               Brand|Year|quarter|ProductSold|
+--------------------+----+-------+-----------+
|2(x)ist men's 3-p...|2016|      2|        1.0|
|2(x)ist men's 3-p...|2017|      1|        1.0|
|                32gb|2015|      1|        4.0|
|                32gb|2016|      1|        2.0|
|                32gb|2017|      2|        2.0|
|                32gb|2017|      3|        2.0|
|                32gb|2017|      1|        2.0|
|                32gb|2018|      1|        4.0|
| 3m (formerly aearo)|2015|      2|        2.0|
| 3m (formerly aearo)|2016|      2|        4.0|
| 3m (formerly aearo)|2016|      3|        8.0|
| 3m (formerly aearo)|2017|      2|        2.0|
| 3m (formerly aearo)|2017|      3|        2.0|
| 3m (formerly aearo)|2018|      1|        4.0|
| 3m (formerly aearo)|2018|      2|        2.0|
|                abba|2015|      2|        4.0|
|                abba|2015|      4|        8.0|
|                abba|2015|      1|     

#3.Find products that generated profit

In [0]:
#a. Top 5 profit margin generating products for a given month. (2015 December)
spark.sql("select ProductCode, sum(TransactionAmount) as Revenue  from amazontable where Year = 2015 and Month =12 group by ProductCode order by Revenue Desc").show(5)

+--------------------+-------+
|         ProductCode|Revenue|
+--------------------+-------+
|cae793d9-04f3-49a...|2192400|
|60b5bb50-d175-41a...|1663200|
|b7e5e050-ec00-4db...| 906192|
|d87f06a9-be14-43a...| 848092|
|960f8ae2-a8b5-41b...| 715456|
+--------------------+-------+
only showing top 5 rows



#4.Brands that need marketing

In [0]:
 #a. Brand wise segregation of best performing categories profits wise
spark.sql("select Brand, Category, sum(TransactionAmount) as Revenue from amazontable where Year > 2000 and Year<2022 group by Brand,Category  order by Brand, Category, Revenue ASC").show()

+--------------------+-----------+-------+
|               Brand|   Category|Revenue|
+--------------------+-----------+-------+
|2(x)ist men's 3-p...|   clothing|   4410|
|                32gb|electronics| 312480|
| 3m (formerly aearo)|   clothing|  73920|
|                abba|   clothing| 268128|
|         abercrombie|   clothing| 280602|
|              abound|   clothing| 228690|
|              acacia|   clothing|4464376|
|            academie|   clothing|1084209|
|                acer|electronics|6084743|
|               acorn|   clothing|4955237|
|               acorn|      shoes| 141120|
|           actiontec|electronics|  85120|
|              active|   clothing| 452256|
|              active|electronics|  92064|
|                adam|   clothing| 807681|
|          adamsbuilt|   clothing|  21168|
|                adan|   clothing| 422800|
|                adar|   clothing|  31640|
|                adee|   clothing|   9240|
|              adeyro|   clothing|   5796|
+----------

In [0]:
#b. Brand wise segregation of least performing categories orders wise
spark.sql("select Brand, Category, sum(TransactionAmount) as Revenue from amazontable where Year > 2000 and Year<2022 group by Brand,Category  order by Brand, Category, Revenue DESC").show()

+--------------------+-----------+-------+
|               Brand|   Category|Revenue|
+--------------------+-----------+-------+
|2(x)ist men's 3-p...|   clothing|   4410|
|                32gb|electronics| 312480|
| 3m (formerly aearo)|   clothing|  73920|
|                abba|   clothing| 268128|
|         abercrombie|   clothing| 280602|
|              abound|   clothing| 228690|
|              acacia|   clothing|4464376|
|            academie|   clothing|1084209|
|                acer|electronics|6084743|
|               acorn|   clothing|4955237|
|               acorn|      shoes| 141120|
|           actiontec|electronics|  85120|
|              active|   clothing| 452256|
|              active|electronics|  92064|
|                adam|   clothing| 807681|
|          adamsbuilt|   clothing|  21168|
|                adan|   clothing| 422800|
|                adar|   clothing|  31640|
|                adee|   clothing|   9240|
|              adeyro|   clothing|   5796|
+----------

#5.Find the products that should be part of the lightning deals.

In [0]:
#a. Find the top five selling products in a given month
spark.sql("select Year,Month, SubCategory, sum(Quantity) as Quantity from amazontable where Year>2000 and Year < 2022 group by Year,Month, SubCategory, Quantity ORDER BY Year, Month").show()

+----+-----+--------------------+--------+
|Year|Month|         SubCategory|Quantity|
+----+-----+--------------------+--------+
|2015|    1|swimsuits & cover...|     1.0|
|2015|    1|           bracelets|     3.0|
|2015|    1|   intimates & sleep|     6.0|
|2015|    1|laptop cases & co...|     2.0|
|2015|    1|            clothing|     1.0|
|2015|    1|               heels|     1.0|
|2015|    1|     jackets & vests|     9.0|
|2015|    1|         men's rings|     1.0|
|2015|    1|     womens costumes|     1.0|
|2015|    1|               flats|     2.0|
|2015|    1|cold weather acce...|     4.0|
|2015|    1|   men's dress shoes|    12.0|
|2015|    1|women's athletic ...|    12.0|
|2015|    1|    petite swimsuits|     2.0|
|2015|    1|     coats & jackets|     4.0|
|2015|    1|              bangle|     4.0|
|2015|    1|             running|     4.0|
|2015|    1|               rings|     6.0|
|2015|    1|             lcd tvs|     4.0|
|2015|    1|plus size activewear|     2.0|
+----+-----

In [0]:
#b. Find the top five products having the highest margin in a given month

spark.sql("select Year, Month, SubCategory, Marginpercentage from "+"(select *,row_number() OVER (PARTITION BY SubCategory,Month Order by Marginpercentage desc ) as rn" + " from amazontable) tmp where rn <= 5 and Year >2010 and Year < 2022 group by Year, Month, SubCategory, Marginpercentage" ).show()

+----+-----+-----------+----------------+
|Year|Month|SubCategory|Marginpercentage|
+----+-----+-----------+----------------+
|2016|    1|accessories|              30|
|2017|    1|accessories|              30|
|2018|    1|accessories|              30|
|2015|    1|accessories|              20|
|2016|    2|accessories|              30|
|2017|    2|accessories|              30|
|2018|    2|accessories|              30|
|2015|    2|accessories|              20|
|2015|    3|accessories|              30|
|2016|    3|accessories|              30|
|2015|    4|accessories|              30|
|2015|    4|accessories|              20|
|2017|    4|accessories|              20|
|2015|    4|accessories|              10|
|2015|    5|accessories|              30|
|2016|    5|accessories|              30|
|2015|    6|accessories|              30|
|2016|    6|accessories|              30|
|2017|    6|accessories|              30|
|2015|    7|accessories|              30|
+----+-----+-----------+----------

In [0]:
#c. Find the five lowest selling products in a given month.
spark.sql("select Year,Month, SubCategory, sum(Quantity) as Quantity from amazontable where Year>2000 and Year < 2022 group by Year,Month, SubCategory, Quantity ORDER BY Quantity ASC limit 5").show(truncate=False)

+----+-----+--------------------+--------+
|Year|Month|SubCategory         |Quantity|
+----+-----+--------------------+--------+
|2015|5    |sweats & hoodies    |1.0     |
|2015|6    |boat shoes          |1.0     |
|2015|3    |hats                |1.0     |
|2015|1    |heels               |1.0     |
|2015|5    |cross-training shoes|1.0     |
+----+-----+--------------------+--------+



#6.Trending brands for given month order wise

In [0]:
spark.sql("select Brand,SubCategory,Year,Month,sum(Quantity) as Quantity from amazontable where Year >2000 and Year <2022 group by Brand, Year, Month, Subcategory order by Month,Quantity desc").show(truncate=False)


+-------+-----------+----+-----+--------+
|Brand  |SubCategory|Year|Month|Quantity|
+-------+-----------+----+-----+--------+
|samsung|mobile     |2017|1    |416.0   |
|samsung|mobile     |2016|1    |402.0   |
|samsung|mobile     |2015|1    |377.0   |
|samsung|mobile     |2018|1    |377.0   |
|nike   |men's shoes|2016|1    |261.0   |
|nike   |men's shoes|2015|1    |240.0   |
|apple  |mobile     |2016|1    |225.0   |
|apple  |mobile     |2015|1    |223.0   |
|nike   |men's shoes|2018|1    |203.0   |
|nokia  |mobile     |2016|1    |198.0   |
|nike   |men's shoes|2017|1    |196.0   |
|blu    |mobile     |2016|1    |195.0   |
|blu    |mobile     |2018|1    |174.0   |
|apple  |mobile     |2018|1    |174.0   |
|nokia  |mobile     |2015|1    |168.0   |
|lg     |mobile     |2016|1    |162.0   |
|blu    |mobile     |2017|1    |156.0   |
|blu    |mobile     |2015|1    |148.0   |
|apple  |mobile     |2017|1    |137.0   |
|lg     |mobile     |2018|1    |135.0   |
+-------+-----------+----+-----+--